In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test, ON_THRESHOLD
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [2]:
tensor = np.load("../2015-5appliances.numpy.npy")
dr = tensor[:, 3]

In [3]:
# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]

In [5]:
threshold = {}
for fold_num in range(5):
    threshold[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        
        sample_list = []
        for i in range(test_gt[fold_num][appliance].shape[0]):
            sample_list = np.append(sample_list, np.mean([x for x in test_gt[fold_num][appliance][i].reshape(1, -1).tolist()[0] if x > ON_THRESHOLD[appliance]]))
        threshold[fold_num][appliance] = 0.1*sample_list

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
def calculate_error(pred, gt, threshold):
    error = {}
    overall = {}
    num_homes = {}
    
    # calculte number of homes in each fold
    for fold_num in range(5):
        num_homes[fold_num] = gt[fold_num]['hvac'].reshape(-1, 1, 112,24).shape[0]
    homes = pd.Series(num_homes).sum()
    
    # for Constantly On appliances: hvac and fridge
#     for appliance in ['hvac', 'fridge']:
#         overall[appliance] = 0
#         error[appliance] = {}
#         for fold_num in range(5):
#             error[appliance][fold_num] = mean_absolute_error(pred[fold_num][appliance].reshape(-1, 24), 
#                                                              gt[fold_num][appliance].reshape(-1, 24))
#             overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
#         overall[appliance] /= homes
        
    # for ON/OFF appliances: dryer, dishwasher and microwave
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        error[appliance] = {}
        overall[appliance] = 0                                                                
        for fold_num in range(5):
            error[appliance][fold_num] = onoff_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                     gt[fold_num][appliance].reshape(-1, 24), threshold[fold_num][appliance])
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
    
    
    return error, overall

In [7]:
def onoff_error(pred, gt, threshold):
    pred = pred.reshape(-1 ,112, 24)
    gt = gt.reshape(-1 ,112, 24)
    homes = pred.shape[0]
    error = {}
    abs_error = np.abs(pred-gt)
    if threshold == None:
        threshold = np.zeros(homes)
    for i in range(homes):
        error_list = [x for x in abs_error[i].reshape(1, -1)[0] if x >= threshold[i]]
        if len(error_list) == 0:
            error[i] = 0
        else:
            error[i] = np.mean([x for x in abs_error[i].reshape(1, -1)[0] if x >= threshold[i]])
#     print(abs_error[i])
#     print(threshold, error)
    
    
    return (pd.Series(error).mean())

## Dataset 1: Artificial Aggregate Dataset

### Individual RNN and TreeRNN
#### Just use the best parameter found before as RNN always predict dryer as zero

In [26]:
dataset = 1
best_rnn_individual = np.load("../code/baseline/result/rnn-individual-param-{}.npy".format(dataset)).item()
best_rnn_tree = np.load("../code/baseline/result/rnn-tree-param-{}.npy".format(dataset)).item()

In [27]:
# get prediction for rnn individual
rnn_individual_pred = {}
rnn_individual_error = {}
for fold_num in range(5):
    rnn_individual_pred[fold_num] = {}
    rnn_individual_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        cell_type = best_rnn_individual[appliance][fold_num]['cell_type']
        hidden_size = best_rnn_individual[appliance][fold_num]['hidden_size']
        num_layers = best_rnn_individual[appliance][fold_num]['num_layers']
        lr = best_rnn_individual[appliance][fold_num]['lr']
        iters = best_rnn_individual[appliance][fold_num]['iters']
        bidirectional = best_rnn_individual[appliance][fold_num]['bidirectional']

        directory = "../code/baseline/rnn-tree/{}/{}/{}/{}/{}/{}/{}/3000/0.0/".format(dataset, fold_num, cell_type,
                                                                                                    hidden_size,
                                                                                                    num_layers,
                                                                                                    bidirectional,
                                                                                                    lr)
        filename = "test-pred-{}-{}-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, dataset, cell_type, hidden_size, num_layers,
                                                                                 bidirectional, lr, appliance)

        full_path = directory + filename
        rnn_individual_pred[fold_num][appliance] = np.asarray(np.load(full_path).item()[iters]).reshape(-1, 1, 112, 24)

In [28]:
# get prediction for rnn tree
rnn_tree_pred = {}
rnn_tree_error = {}
for fold_num in range(5):
    # get prediction for rnn tree
    rnn_tree_pred[fold_num] = {}
#     rnn_tree_error[fold_num] = {}
    cell_type = best_rnn_tree[fold_num]['cell_type']
    hidden_size = best_rnn_tree[fold_num]['hidden_size']
    num_layers = best_rnn_tree[fold_num]['num_layers']
    lr = best_rnn_tree[fold_num]['lr']
    iters = best_rnn_tree[fold_num]['iters']
    bidirectional = best_rnn_tree[fold_num]['bidirectional']
    order = best_rnn_tree[fold_num]['order']
    directory = "../code/baseline/rnn-tree/{}/{}/{}/{}/{}/{}/{}/3000/0.0/".format(dataset, fold_num, cell_type,
                                                                                                hidden_size,
                                                                                                num_layers,
                                                                                                bidirectional,
                                                                                                float(lr))
    filename = "test-pred-{}-{}-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, dataset, cell_type, hidden_size, num_layers,
                                                                             bidirectional, float(lr), str(order)[2:-2])

    full_path = directory + filename
    rnn_pred = np.asarray(np.load(full_path).item()[iters])
    

    for idx, appliance in enumerate(order):
        rnn_tree_pred[fold_num][appliance] = rnn_pred[idx].reshape(-1, 1, 112,24)

In [29]:
pd.DataFrame(calculate_error(rnn_individual_pred, test_gt, threshold)[0])

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  import sys


,dr,dw,fridge,hvac,mw
0,1196.674072,328.679178,35.522373,395.690890,54.993966
1,1212.599873,377.315883,40.464959,514.588107,49.839184
2,1194.534857,365.774494,33.686852,472.679701,40.408460
3,1264.585252,358.667728,37.671813,521.152759,42.496978
4,1235.777338,299.161093,34.055233,389.721292,59.691091


In [30]:
pd.DataFrame(calculate_error(rnn_tree_pred, test_gt, threshold)[0])

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  import sys


,dr,dw,fridge,hvac,mw
0,1196.672115,328.678510,34.257107,382.245507,54.908629
1,1212.302904,377.381632,40.610597,511.111714,48.244586
2,1197.009230,365.774494,31.512746,462.168769,40.321809
3,1264.523766,358.806769,37.284580,523.286864,42.528929
4,1235.772468,299.160486,35.801542,403.642787,59.828349


## Individual CNN and TreeCNN

In [51]:
dataset = 3

In [52]:
cnn_individual_valid_pred = {}
gid = 0
c = 0
for fold_num in range(5):
    cnn_individual_valid_pred[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        cnn_individual_valid_pred[fold_num][appliance] = {}
        for lr in [0.001, 0.01, 0.1]:
            cnn_individual_valid_pred[fold_num][appliance][lr] = {}
            for iters in [200000]:
            
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "valid-pred-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                    c += 1
                else:
#                     continue
                    k = np.load(full_path).item()
                    for it in range(1000, 200000+1, 1000):
                        cnn_individual_valid_pred[fold_num][appliance][lr][it] = k[it][0]
print(c)

0


In [53]:
np.save("../code/baseline/cnn_individual_valid_pred_3.npy", cnn_individual_valid_pred)

In [32]:
# cnn_individual_best_param = {}
# for fold_num in range(5):
#     cnn_individual_best_param[fold_num] = {}
#     for appliance in ['hvac', 'fridge']:
#         cnn_individual_best_param[fold_num][appliance] = {}
#         min_error = np.inf
#         for lr in [0.001, 0.01, 0.1]:
#             for it in range(1000, 200000+1, 1000):
#                 error = mean_absolute_error(cnn_individual_valid_pred[fold_num][appliance][lr][it].reshape(-1, 24), valid_gt[fold_num][appliance].reshape(-1, 24))
#                 if error < min_error:
#                     cnn_individual_best_param[fold_num][appliance]['lr'] = lr
#                     cnn_individual_best_param[fold_num][appliance]['iters'] = it
#                     min_error = error

In [33]:
cnn_individual_best_param = {}

for fold_num in range(5):
    cnn_individual_best_param[fold_num] = {}

    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        cnn_individual_best_param[fold_num][appliance] = {}
        min_error = np.inf
        for lr in [0.001, 0.01, 0.1]:
            for it in range(1000, 200000+1, 1000):
                error = onoff_error(cnn_individual_valid_pred[fold_num][appliance][lr][it].reshape(-1, 24), valid_gt[fold_num][appliance].reshape(-1, 24), threshold[fold_num][appliance])
                if error < min_error:
                    cnn_individual_best_param[fold_num][appliance]['lr'] = lr
                    cnn_individual_best_param[fold_num][appliance]['iters'] = it
                    min_error = error

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  import sys


In [34]:
np.save("../code/baseline/result/cnn-individual-new-param-{}-per-home.npy".format(dataset), cnn_individual_best_param)

In [35]:
gid = 0
srv = 1
c = 0
cmd = {}
cnn_tree_valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    cnn_tree_valid_pred[fold_num] = {}
    for lr in [0.01]:
        cnn_tree_valid_pred[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
            
            if order[0] == 'hvac':
                continue
            
            cnn_tree_valid_pred[fold_num][lr][order] = {}


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-pred-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
                print(line)

                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                k = np.load(full_path).item()
                for it in range(1000, 20001, 1000):
                    cnn_tree_valid_pred[fold_num][lr][order][it] = {}
                    for idx, appliance in enumerate(order):
                        cnn_tree_valid_pred[fold_num][lr][order][it][appliance] = k[it][idx]

print(c)                               

0


In [36]:
cnn_tree_best_param = {}
for fold_num in range(5):
    cnn_tree_best_param[fold_num] = {}
    min_error = np.inf
    for lr in [0.01]:
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
            if order[0] == 'hvac':
                continue
            for it in range(1000, 20001, 1000):
                error = 0
                for idx, appliance in enumerate(order):
#                     if appliance in ['hvac', 'fridge']:
#                         error += mean_absolute_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
#                                                     valid_gt[fold_num][appliance].reshape(-1, 24))
#                     else:
                    error += onoff_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                valid_gt[fold_num][appliance].reshape(-1, 24), threshold[fold_num][appliance])
                if error < min_error:
                    min_error = error
                    cnn_tree_best_param[fold_num]['lr'] = lr
                    cnn_tree_best_param[fold_num]['order'] = order
                    cnn_tree_best_param[fold_num]['iters'] = it
                
                    

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  import sys


In [37]:
np.save("../code/baseline/result/cnn-tree-new-param-{}-per-home.npy".format(dataset), cnn_tree_best_param)

In [43]:
# load the best for cnn model
dataset = 3
best_cnn_individual = np.load("../code/baseline/result/cnn-individual-new-param-{}-per-home.npy".format(dataset)).item()
best_cnn_tree = np.load("../code/baseline/result/cnn-tree-new-param-{}-per-home.npy".format(dataset)).item()

In [44]:
# get prediction for cnn individual
cnn_individual_pred = {}
for fold_num in range(5):
    cnn_individual_pred[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        
        lr = best_cnn_individual[fold_num][appliance]['lr']
        iters = best_cnn_individual[fold_num][appliance]['iters']

        directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num,lr)
        filename = "test-pred-[\'{}\'].npy".format(appliance)

        full_path = directory + filename
        cnn_individual_pred[fold_num][appliance] = np.asarray(np.load(full_path).item()[iters]).reshape(-1, 1, 112, 24)

In [45]:
# get prediction for cnn tree
cnn_tree_pred = {}
for fold_num in range(5):
    cnn_tree_pred[fold_num] = {}
        
    lr = best_cnn_tree[fold_num]['lr']
    iters = best_cnn_tree[fold_num]['iters']
    order = best_cnn_tree[fold_num]['order']

    o = "\', \'".join(str(x) for x in order)
    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
    filename = "test-pred-[\'{}\'].npy".format(o)

    full_path = directory + filename
    cnn_pred = np.asarray(np.load(full_path).item()[iters])
    
    for idx, appliance in enumerate(order):
        cnn_tree_pred[fold_num][appliance] = cnn_pred[idx].reshape(-1, 1, 112,24)
    

In [46]:
new_error = {}
new_error['Individual RNN'] = calculate_error(rnn_individual_pred, test_gt, threshold)[1]
new_error['Tree RNN'] = calculate_error(rnn_tree_pred, test_gt, threshold)[1]
new_error['Individual CNN'] = calculate_error(cnn_individual_pred, test_gt, threshold)[1]
new_error['Tree CNN'] = calculate_error(cnn_tree_pred, test_gt, threshold)[1]

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  import sys


In [47]:
pd.DataFrame(new_error)

,Individual CNN,Individual RNN,Tree CNN,Tree RNN
dr,1030.008292,1219.971131,909.256618,1220.406332
dw,337.914037,346.419830,333.676081,346.459694
fridge,37.347658,36.292503,39.160847,35.874204
hvac,408.062568,458.864477,409.262905,456.286019
mw,40.917312,49.438639,40.151385,49.107279


In [178]:
pd.DataFrame(calculate_error(cnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,310.504847,96.143893,31.031232,180.355370,10.083670
1,523.179778,248.991752,36.696303,138.862061,13.908121
2,533.766609,288.867558,35.966973,147.254607,8.970281
3,231.743222,157.640404,35.664400,181.893883,7.568580
4,281.008578,104.027361,35.186277,170.603921,17.477861


In [124]:
pd.Series(calculate_error(rnn_individual_pred, test_gt, threshold)[1])

dr        1195.133984
dw         345.890323
fridge      27.901909
hvac       130.851865
mw          52.699242
dtype: float64

In [125]:
pd.DataFrame(calculate_error(rnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,1113.821826,344.085752,32.386288,90.562519,34.026531
1,835.608889,352.366163,30.798445,109.945964,34.165309
2,967.544036,361.354331,25.768721,101.670040,29.484331
3,953.552660,369.728983,28.358526,131.728544,28.449334
4,862.044121,300.189794,28.586311,99.395924,69.898952


In [126]:
pd.DataFrame(calculate_error(cnn_individual_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,418.190039,339.861556,28.517904,175.616321,35.287017
1,582.483109,336.303458,34.600568,135.859949,40.802098
2,635.744956,355.839423,27.424491,160.653717,31.898192
3,558.069406,339.532500,32.630367,163.393771,34.496536
4,532.558061,276.159278,30.406227,151.497976,53.638239


In [127]:
pd.DataFrame(calculate_error(cnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,380.361517,221.357761,40.789443,183.317879,32.954544
1,654.570013,308.036766,36.696303,138.862061,38.031077
2,832.800544,348.930867,35.966973,147.254607,29.218336
3,396.865739,338.646366,45.409241,176.136952,34.278379
4,337.580422,191.665894,39.705588,171.053769,57.125152
